In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

In [2]:
from datasets import load_dataset

dataset = load_dataset("timdettmers/openassistant-guanaco", split="train")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/timdettmers--openassistant-guanaco-c21e85fd8b1a6952/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "tiiuae/falcon-7b"


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config= BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    ),
    trust_remote_code=True
)
model.config.use_cache = False

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token

In [8]:
from peft import LoraConfig

peft_config= LoraConfig(
    lora_alpha= 16,
    lora_dropout= 0.1,
    r = 64,
    bias= "none",
    task_type= "CAUSAL_LM",
    target_modules= [
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

In [9]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_arguments = TrainingArguments(
    output_dir= "./results",
    per_device_train_batch_size= 4,
    gradient_accumulation_steps= 4,
    optim= "paged_adamw_32bit",
    save_steps= 10,
    logging_steps= 10,
    learning_rate= 2e-4,
    fp16=True,
    max_grad_norm= 0.3,
    max_steps= 500,
    warmup_ratio= 0.03,
    group_by_length= True,
    lr_scheduler_type= "constant",
    gradient_checkpointing= True,
)

max_seq_length= 512

trainer= SFTTrainer(
    model= model,
    train_dataset= dataset,
    peft_config= peft_config,
    dataset_text_field= "text",
    max_seq_length= max_seq_length,
    tokenizer= tokenizer,
    args= training_arguments,
)

  0%|          | 0/10 [00:00<?, ?ba/s]

In [10]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
trainer.train()